# Week 9: Bisector Area of 3D Mesh
**Target**: 

In [ ]:
# Packages
import numpy as np
import random
import matplotlib.pyplot as plt

# Self-defined functions
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))
from util import util

# Visualization
import pyvista as pv
from pyvista import examples
from util import util_3d

In [ ]:
points = np.array([[0, 1, 0], [-1, -1, -2], [1, 0, -1]])
faces = np.array([3, 0, 1, 2])
util_3d.make_clockwise_3d(points)
mesh = pv.PolyData(points, faces)

plane = pv.Plane(center=(0, 0.5, 0), direction=(0, 1, 0))

center = mesh.points.mean(axis=0)
normal = mesh.face_normals[0]

plotter = pv.Plotter()
plotter.add_mesh(mesh, color="lightblue", show_edges=True)
plotter.add_mesh(plane, color="yellow", show_edges=True)
plotter.view_vector(vector=normal, viewup=[0, 1, 0])
plotter.show_axes()
plotter.show()

In [51]:
def cross_product_derivative(A, B, A_dot, B_dot):
    C_dot = np.cross(A_dot, B) + np.cross(A, B_dot)
    return C_dot


def derivative_of_magnitude(v, dv_dt):
    v_norm = np.linalg.norm(v)
    if v_norm == 0:
        return 0
    else:
        df_dt = np.dot(v, dv_dt) / v_norm
        return df_dt

In [17]:
len(sphere.points)

842

In [18]:
test = problem_mesh_3D(sphere.points, sphere.faces.reshape((-1,4))[:, 1:4], sphere.face_normals)

In [122]:
test = problem_mesh_3D(mesh.points, mesh.faces.reshape((-1,4))[:, 1:4], mesh.face_normals)

NameError: name 'util_3dmake_clockwise_3d' is not defined

In [123]:
test.train_one_round()

S:  4.242640687119285
The loss is:  10.125000305182148
The enclosing area is:  4.242640687119285
一半面积：  1.0606601238250732
dL/dD is:  -876.8720576880601
D is:  55.37450358132132


Widget(value="<iframe src='http://localhost:57778/index.html?ui=P_0x1b642dd07d0_8&reconnect=auto' style='width…

In [121]:
class problem_mesh_3D:
    def __init__(self, points, faces, normals, n=[0,1,0], ratio=0.5, learning_rate=0.05, num_iteration=1000):
        if len(points) < 3:
            A = np.array([6,9])
            B = np.array([10,14])
            C = np.array([8,28])
            self.vertices = np.array([A, B, C])
        else:
            self.vertices = points
        self.faces = faces
        self.normals = normals
        util_3dmake_clockwise_3d(self.vertices)
        
        # Initialize variable "D" and define the plane
        D = 0.5
        self.plane=np.array(n + [D])
        self.N=n
    
        # Area realted
        self.ratio = ratio
        self.compute_mesh_area()
        
        # Learning realted
        self.lr = learning_rate
        self.num_iteration = num_iteration
        
    def compute_mesh_area(self):
        self.area = 0
        for face in self.faces:
            index_A, index_B, index_C = face
            A = self.vertices[index_A]
            B = self.vertices[index_B]
            C = self.vertices[index_C]
            self.area += util.triangle_area(A, B, C)
        return self.area
    
    def forward_propagate(self):
        self.S = 0
        self.dS_dD = 0
        
        for idx, face in enumerate(self.faces):
            index_A, index_B, index_C = face
            A = points[index_A]
            B = points[index_B]
            C = points[index_C]
            
            # Edges
            line_AC = util_3d.line_3D(A, C-A)
            line_AB = util_3d.line_3D(A, B-A)
            line_BA = util_3d.line_3D(B, A-B)
            line_BC = util_3d.line_3D(B, C-B)
            line_CB = util_3d.line_3D(C, B-C)
            line_CA = util_3d.line_3D(C, C-A)
            
            if not np.isclose(np.dot(line_AC.n, self.N), 0) and not np.isclose(np.dot(line_AB.n, self.N), 0):
                # The area
                AC = line_AC.intersection_plane_t(self.plane) * line_AC.n
                AB = line_AB.intersection_plane_t(self.plane) * line_AB.n
                # Directed area
                SA = np.cross(AC, AB)
                # The gradients
                # dt/dD
                dAC_dD = line_AC.n / np.dot(line_AC.n, self.N)
                dAB_dD = line_AB.n / np.dot(line_AB.n, self.N)
                dSA_dD = cross_product_derivative(AC, AB, dAC_dD, dAB_dD)
                
            if not np.isclose(np.dot(line_BA.n, self.N), 0) and not np.isclose(np.dot(line_BC.n, self.N), 0):
                # The area
                BA = line_BA.intersection_plane_t(self.plane) * line_BA.n
                BC = line_BC.intersection_plane_t(self.plane) * line_BC.n
                # Directed area
                SB = np.cross(BA, BC)
                # The gradients
                # dt/dD
                dBA_dD = line_BA.n / np.dot(line_BA.n, self.N)
                dBC_dD = line_BC.n / np.dot(line_BC.n, self.N)
                dSB_dD = cross_product_derivative(BA, BC, dBA_dD, dBC_dD)
                
            if not np.isclose(np.dot(line_CB.n, self.N), 0) and not np.isclose(np.dot(line_CA.n, self.N), 0):
                # The area
                CB = line_CB.intersection_plane_t(self.plane) * line_CB.n
                CA = line_CA.intersection_plane_t(self.plane) * line_CA.n
                # Directed area
                SC = np.cross(CB, CA)
                # The gradients
                # dt/dD
                dCB_dD = line_BA.n / np.dot(line_CB.n, self.N)
                dCA_dD = line_BC.n / np.dot(line_CA.n, self.N)
                dSC_dD = cross_product_derivative(CB, CA, dCB_dD, dCA_dD)

            self.S += np.linalg.norm(SA + SB + SC)
            self.dS_dD += derivative_of_magnitude(SA + SB + SC, dSA_dD+dSB_dD+dSC_dD)
            print("S: ", self.S)
    
    def loss_MSE(self):
        return np.power(self.S-self.ratio*self.area, 2)
    
#     def forward_propagation(self):
        
    def backward_propagation(self):
        dL_dS = 2*self.S-2*self.ratio*self.area
        self.dL_dD=dL_dS*self.dS_dD
        
    def update_parameters(self):
        A, B, C, D = self.plane
        D -= self.lr * self.dL_dD
        self.plane = np.array([A, B, C, D])
        
    def visualize(self):
        A, B, C, D = self.plane
        plane = pv.Plane(center=(0, D, 0), direction=(0, 1, 0))
        
        plotter = pv.Plotter()
        plotter.add_mesh(mesh, color="lightblue", show_edges=True)
        plotter.add_mesh(plane, color="yellow", show_edges=True)
        plotter.view_vector(vector=normal, viewup=[0, 1, 0])
        plotter.show_axes()
        plotter.show()
        
    def train_one_round(self):
        self.forward_propagate()
        print("The loss is: ", self.loss_MSE())
        self.backward_propagation()
        self.update_parameters()
        
        print("The enclosing area is: ", self.S)
        print("一半面积： ", self.ratio*self.area)
        print("dL/dD is: ", self.dL_dD)
        print("D is: ", self.plane[3])
        self.visualize()
        
    def train(self):
        for i in range(self.num_iteration):
            self.forward_propagate()
            self.backward_propagation()
            self.update_parameters()
        
        print("The loss is: ", self.loss_MSE())
        print("The enclosing area is: ", self.S)
        self.visualize()

In [4]:
sphere = pv.Sphere()

pl = pv.Plotter()
pl.add_mesh(sphere, show_edges=True, color="white")
pl.add_points(sphere.points, color="red", point_size=5)
pl.show_axes()
pl.show()

Widget(value="<iframe src='http://localhost:57778/index.html?ui=P_0x1b631014650_1&reconnect=auto' style='width…

In [ ]:
test = problem_mesh_3D(sphere.points, sphere.faces.reshape((-1,4))[:, 1:4], sphere.face_normals)